# Stats Review M1

### Requirements

In [1]:
from scipy import stats
import numpy as np
import plotly.express as px
import pandas as pd
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import math

In [2]:
X_mean = np.array(
    [
        [
            5.5,
            5,
            3.25
        ]
    ]
).reshape(3, 1)

print(X_mean)

print(X_mean.shape)

[[5.5 ]
 [5.  ]
 [3.25]]
(3, 1)


In [3]:
XXT_mean = np.matmul(
    X_mean,
    X_mean.T
)

print(XXT_mean)

print(XXT_mean.shape)

[[30.25   27.5    17.875 ]
 [27.5    25.     16.25  ]
 [17.875  16.25   10.5625]]
(3, 3)


In [4]:
X = np.array(
    [
        [
            8,
            2,
            3,
            9
        ],
        [
            4,
            8,
            1,
            7
        ],
        [
            7,
            1,
            1,
            4
        ]
    ]
)

print(X)

print(X.shape)

[[8 2 3 9]
 [4 8 1 7]
 [7 1 1 4]]
(3, 4)


In [5]:
XXT = None
for col in X.T:
    col_T = np.array([col])
    print(col_T.shape)
    print(col_T)
    XXT = np.matmul(col_T.T, col_T) if XXT is None else (XXT + np.matmul(col_T.T, col_T))
    print(XXT)

(1, 3)
[[8 4 7]]
[[64 32 56]
 [32 16 28]
 [56 28 49]]
(1, 3)
[[2 8 1]]
[[68 48 58]
 [48 80 36]
 [58 36 50]]
(1, 3)
[[3 1 1]]
[[77 51 61]
 [51 81 37]
 [61 37 51]]
(1, 3)
[[9 7 4]]
[[158 114  97]
 [114 130  65]
 [ 97  65  67]]


In [6]:
S = (XXT/X.shape[1]) - XXT_mean

print(S)

print(S.shape)

[[9.25   1.     6.375 ]
 [1.     7.5    0.    ]
 [6.375  0.     6.1875]]
(3, 3)


In [7]:
I1 = np.array(
    [
        [
            1, 1, 1
        ]
    ]
).reshape(3, 1)

print(I1)

print(I1.shape)

[[1]
 [1]
 [1]]
(3, 1)


In [8]:
H = np.eye(3) - (np.matmul(I1, I1.T)/3)

In [9]:
H

array([[ 0.66666667, -0.33333333, -0.33333333],
       [-0.33333333,  0.66666667, -0.33333333],
       [-0.33333333, -0.33333333,  0.66666667]])

In [10]:
np.matmul(H, H)

array([[ 0.66666667, -0.33333333, -0.33333333],
       [-0.33333333,  0.66666667, -0.33333333],
       [-0.33333333, -0.33333333,  0.66666667]])

In [11]:
H**2

array([[0.44444444, 0.11111111, 0.11111111],
       [0.11111111, 0.44444444, 0.11111111],
       [0.11111111, 0.11111111, 0.44444444]])

In [12]:
0.6**2

0.36

In [23]:
x = np.array(
    [
        [
            2, -1, -2
        ]
    ]
).reshape(3, 1)

print(x)

print(x.shape)

[[ 2]
 [-1]
 [-2]]
(3, 1)


In [42]:
np.matmul(np.matmul(H, H), x)

array([[ 2.33333333],
       [-0.66666667],
       [-1.66666667]])

In [37]:
np.matmul(H**2, x)

array([[ 0.55555556],
       [-0.44444444],
       [-0.77777778]])

In [4]:
reg = LinearRegression().fit(
    X= (
        frame
        .drop(columns=["LogPlanetMass"])
    ),
    y=frame.LogPlanetMass
    )

In [5]:
reg.coef_

array([ 1.40214538, -0.14099818, -1.59951364, -0.95610919, -0.46176234])

In [6]:
reg.intercept_

0.15379302910641224

In [32]:
frame.insert(loc=0, column="intercept", value=1)

X = (
    frame
    .drop(columns=["LogPlanetMass"])
    .to_numpy()
)

In [55]:
X.shape

(30, 6)

In [56]:
XX_inv = np.linalg.inv(np.matmul(X.T, X))

In [57]:
XX_inv.shape

(6, 6)

In [60]:
XX_inv_sqrt = np.vectorize(np.sqrt)(XX_inv)

/Users/administrator/Desktop/Analysis /Code/.venv/lib/python3.9/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: invalid value encountered in sqrt
  outputs = ufunc(*inputs)


In [62]:
epsi = np.diagonal(XX_inv_sqrt)

In [63]:
frame['Predicted'] = reg.predict(X=(
    frame
    .drop(
        columns=['intercept', "LogPlanetMass"]
    )
))

In [64]:
frame['Residuals'] = frame.LogPlanetMass - frame.Predicted

In [67]:
frame = frame.assign(
    residuals_sqrt = lambda M: M.Residuals.map(lambda u: u**2),
)

In [72]:
sigma_sqrt = 1/(N - 4) * frame.sum(axis=0)['residuals_sqrt']

In [73]:
sigma_epsi = np.sqrt(sigma_sqrt) * epsi

In [81]:
sigma_epsi

array([1.33450155, 0.2752293 , 0.34567635, 1.20149093, 1.07311457,
       0.35697812])

In [79]:
coeffs = np.array([reg.intercept_] + list(reg.coef_))

In [82]:
coeffs

array([ 0.15379303,  1.40214538, -0.14099818, -1.59951364, -0.95610919,
       -0.46176234])

In [80]:
coeffs/sigma_epsi

array([ 0.1152438 ,  5.09446261, -0.40789075, -1.331274  , -0.89096655,
       -1.29353121])

In [83]:
def p_from_t_test(val:float) -> float:
    return stats.t.sf(np.abs(val), 25)*2

In [84]:
np.vectorize(p_from_t_test)(coeffs/sigma_epsi)

array([9.09172545e-01, 2.91944715e-05, 6.86826076e-01, 1.95112131e-01,
       3.81441989e-01, 2.07651956e-01])